### Charging Profile

In [1]:
from scipy.optimize import fsolve
import numpy as np


def cv_pwr(t: float, pm: float, k: float) -> float:
    return pm * np.exp(-k * t)


def cv_eng(t2, t1, pm, k) -> float:
    return (-1 / k) * (cv_pwr(t2, pm, k) - cv_pwr(t1, pm, k))


def cv_pwr_avg(t2, t1, pm, k) -> float:
    return cv_eng(t2, t1, pm, k) / (t2 - t1)


def charge_profile(
    soc: float,
    d_c: float,
    cap: float = 70,
    ev_mpi: float = 7.36,
    k: float = 0.1,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    cv : float
        Capacity at which the charging curve flattens in %.
    cc : float
        Capacity at which the charging curve starts in %.

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cv = 0.8 * cap
    dc = d_c * cap

    # current charge
    c0 = soc * cap
    p0 = ev_mpi
    t0 = c0 / p0

    # first part of charge < 80% of cap
    c1 = max(min(dc, cv) - min(c0, cv), 0)  # charge [kWh] required
    p1 = 0 if c1 == 0 else ev_mpi  #          power [kW] required
    t1 = 0 if c1 == 0 else c1 / p1   #        time [h] required

    # second part of charge > 80% of cap
    c2 = max(max(dc, cv) - max(c0, cv), 0)  # charge [kWh] required
    p2 = 0  #                                 initial power [kW] required
    t2 = 0  #                                 initial time [h] required

    # Define the function for the given equation with specific ta, pm and k
    def zero_for_E(t2, t1, pm, k, E) -> float:
        return cv_eng(t2, t1, pm, k) - E

    # dtermime time and power for 2nd part of charge CV
    if c2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=zero_for_E, x0=t1, args=(0, ev_mpi, k, c2), xtol=1e-3)

        t2 = solution[0]  # time [h] required 2nd part CV
        p2 = c2 / t2  #     power [kW] required 2nd part CV

    # e2 = 0 if t2 == 0 else float(equation(t2))
    e2 = 0 if t2 == 0 else float(zero_for_E(t2=t2, t1=0, pm=ev_mpi, k=k, E=c2))
    t3 = t1 + t2 + abs(np.random.normal(0, 1))

    return {
        "params": {"soc": soc, "d_c": d_c, "cap": cap, "ev_mpi": ev_mpi, "k": k},
        "phase0": {"c0": c0, "t": t0          , "p": p0},
        "phase1": {"c1": c1, "t": t1          , "p": p1},
        "phase2": {"c2": c2, "t": t1 + t2     , "p": p2},
        "phase3": {"c3": 0 , "t": t1 + t2 + t3, "p": 0} ,        
        "result": {"fc": c0 + c1 + c2 + e2, "dc": dc, "rm": dc - (c0 + c1 + c2 + e2)},
        "tslots": {"t1": t1, "t2": t2, "t3": t3},
    }

In [2]:
ev_mpi = 7.36
soc = 0.47
d_c = 1
cap = 70
k = 0.075

# Calculate the charging profile
# https://www.homechargingstations.com/ev-charging-time-calculator/
cp = charge_profile(soc, d_c, cap, ev_mpi, k)


def myprint(cp):
    print(
        round(cp["params"]["soc"], 2),
        round(cp["params"]["d_c"], 2),
        cp["phase1"],
        cp["phase2"],
    )  # , cp["tslots"]["t3"])


print("\nFunction results\n")
print(cp)
# myprint(cp)

# for soc in np.arange(0.1, 1.1, 0.1):
#     for d_c in np.arange(0.1, 1.1, 0.1):
#         cp = charge_profile(soc, d_c, cap, ev_mpi, k)
#         myprint(cp)


Function results

{'params': {'soc': 0.47, 'd_c': 1, 'cap': 70, 'ev_mpi': 7.36, 'k': 0.075}, 'phase0': {'c0': 32.9, 't': 4.470108695652174, 'p': 7.36}, 'phase1': {'c1': 23.1, 't': 3.1385869565217392, 'p': 7.36}, 'phase2': {'c2': 14.0, 't': 5.190910372132185, 'p': 6.8215369436964775}, 'phase3': {'c3': 0, 't': 12.342400834129817, 'p': 0}, 'result': {'fc': 69.99999999857985, 'dc': 70, 'rm': 1.420147555108997e-09}, 'tslots': {'t1': 3.1385869565217392, 't2': 2.0523234156104464, 't3': 7.151490461997632}}


$$
\int_{a}^{b} P_{\text{max}} \times e^{-k \times (t - t_0)} \, dt = 
-\frac{P_{\text{max}}}{k} \left[ e^{-k \times (b - t_0)} - e^{-k \times (a - t_0)} \right]

$$


In [3]:

import pandas as pd

cp1 = charge_profile(0.47, 0.75, cap, ev_mpi, k)
cp2 = charge_profile(0.82, 1.00, cap, ev_mpi, k)

data = [
    cp1['tslots'],
    cp2['tslots']
]
   
df = pd.DataFrame(data)

# Reshape the DataFrame to a single column and sort it
df = df.melt(var_name='time', value_name='value')
df = df.sort_values(by='value')

print(df)
pt = df['value'].unique().tolist()[0:4]

print (pt)
print(type(pt))



  time     value
1   t1  0.000000
2   t2  0.000000
3   t2  1.832279
5   t3  2.661728
0   t1  2.663043
4   t3  4.002568
[0.0, 1.8322791179846367, 2.661727708583718, 2.6630434782608696]
<class 'list'>


In [5]:

import pandas as pd

cp1 = charge_profile(0.47, 0.75, cap, ev_mpi, k)
# print(cp1)
cp2 = charge_profile(0.82, 1.00, cap, ev_mpi, k)
# print(cp2)
cp3 = charge_profile(0.00, 0.90, cap, ev_mpi, k)
# print(cp3)


data = [
    {'ev': "ev1", 't': cp1['phase1']['t'], 'p': cp1['phase1']['p']},
    {'ev': "ev1", 't': cp1['phase2']['t'], 'p': cp1['phase2']['p']},    
    {'ev': "ev1", 't': cp1['phase3']['t'], 'p': cp1['phase3']['p']},        
    {'ev': "ev2", 't': cp2['phase1']['t'], 'p': cp2['phase1']['p']},
    {'ev': "ev2", 't': cp2['phase2']['t'], 'p': cp2['phase2']['p']},    
    {'ev': "ev2", 't': cp2['phase3']['t'], 'p': cp2['phase3']['p']},
    {'ev': "ev3", 't': cp3['phase1']['t'], 'p': cp3['phase1']['p']},
    {'ev': "ev3", 't': cp3['phase2']['t'], 'p': cp3['phase2']['p']},    
    {'ev': "ev3", 't': cp3['phase3']['t'], 'p': cp3['phase3']['p']},    
]
   
df = pd.DataFrame(data)

# Reshape the DataFrame to a single column and sort it
# df = df.sort_values(by='t')
# print(df)

# Remove all records where 't' is 0
df = df.loc[df['t'] != 0]
# Keep only the first 10 columns or fewer if the DataFrame has fewer than 10 columns
df = df.iloc[:, :min(10, df.shape[1])]
# print(df)

def fill_na_with_last_val(row):
    last_val = None
    for col in reversed(row.index):
        if pd.isna(row[col]):
            if last_val is not None:
                row[col] = last_val
        else:
            last_val = row[col]
    return row

# df_pivot = df.pivot(index='ev', columns='t', values='p')
df_pivot = df.pivot_table(index='ev', columns='t', values='p', aggfunc='sum')
# Replace NaN values in the last column with 0
# df_pivot.iloc[:, -1] = df_pivot.iloc[:, -1].fillna(0)

df_pivot = df_pivot.apply(fill_na_with_last_val, axis=1)
print("\n")
print(df_pivot)

column_names = df_pivot.columns.tolist()


slots = [j-i for i, j in zip(column_names[:-1], column_names[1:])]
slots.insert(0, column_names[0])


print(f"pt: {slots}")





t    1.832279   2.663043   4.765150   6.776911   7.608696   8.595408   \
ev                                                                      
ev1   7.360000       7.36       0.00       0.00        NaN        NaN   
ev2   6.876682       0.00       0.00        NaN        NaN        NaN   
ev3   7.360000       7.36       7.36       7.36       7.36   7.094263   

t    18.142102  
ev              
ev1        NaN  
ev2        NaN  
ev3        0.0  
pt: [1.8322791179846367, 0.8307643602762329, 2.1021064564528333, 2.011761273625744, 0.8317844438344659, 0.986712834224126, 9.546693049383503]
